## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

Ja-Chan Lu: Updated research question, updated data after commentary on data section from project proposal.

Tianlin Situ:

Julian Luan:

Tony Zhang:

## Research Question

Updates to Research Question:

**Defining "Eras":**

***Era 1 (1990-2000 seasons):*** Big-man dominated era where most points were scored within close distances to the basket as opposed to three-pointers (much further away from the basket).

***Era 2 (2001-2013 seasons):*** League begins to shy away from traditional post-up and big-men due to the removal of the "illegal defense" which demanded players must be glued to the player they are defending at all times. With the abolishment of that rule, teams were allowed to play zone defense, which allowed players to occupy space on the court and prevent the other team from taking it instead of solely sticking to the player. In zone defenses, teams could theoretically just sufforcate the paint with their players and prevent anyone from making their way into the basket, taking away points from closer to the basket. To counter this, teams began to space out their players in the court, in attempts to work with the little space they have in the perimters to score from further away the basket. 

***Era 3 (2014-Present seasons):*** Teams analyzed shot attempts throughout their seasons and come to the realization that mid-range shots just are not as efficient as shooting 3-pointers. The chances of missing a mid-range and a three-pointer are the same, but three-pointers reward more points than a mid-range, meaning teams would much rather shoot 3's going forward. Teams now push all their players to attempt shooting 3-pointers, rather than mid-range jumpers, further increasing the volume of 3-pointers in the league.

**Editting Reserach Question:**

***New Research Question:*** How has the correlation between height and 3-point attempts changed across select NBA eras? Furthermore, we can then pursue another question: Is the increase in 3-point shooting volume only apply to taller positions (power forwards and centers) or does this affect all positions equally?

## Data

### Update to Data Proposal Section

Actual Dataset Description:
Our dataset will be from swar/nba_api python package, which is an open-source client that gives access to the official data API's on NBA.com. The package is widely used and reliable for accessing NBA data, as seen with their 3,400 GitHub stars. 

As mentioned earlier, the data comes directly from NBA.com's database that gives access to player statistics. For this project, we will utilize the playercareerstats endpoint which keeps records of the player information needed, starting from the 1990 NBA season up until the present day.

For our analysis, we will sample 3-4 datasets (seasons) from each era defined above (1990-2000, 2001-2013, and 2014-present day. Under the assumption that each NBA season has around 500 players, we will have roughly 6000 total observations, where each observation contains a player's id, team_id, height, wingspan, position, points, three_point_attempts, games_played, and minutes_played.

The data in the nba_api originates from official NBA box scores and logs, that were maintained by the league since the NBA was created. During 1990s-2000 era, the statistics were recorded by officials manually whereas 2001-present day, the data was logged and recorded by automated system. Whether it was automated or manually recorded, the data is still reliable and truthful as the NBA continues to verify recorded statistics both during and after games.

Limitations to the data:
1. Records of heights: Throughout all eras, players are to self-report height, and oftentimes may be rounded up/down, rather than 100% truthful.
2. Position Labels: Although all players are given one specific position on paper, their roles on the court often change situationally, meaning no player is truly confined to one role throughout the whole season.
3. In order to ensure accuracy, players who have less than 41 games_played or 10 minutes_played will not be considered in the dataset to prevent skewing the analysis as they will most likely have exceptionally high/low statistics that do not truly reflect their abilities as a player. 

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [ ]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/airline-safety/airline-safety.csv', 'filename':'airline-safety.csv'},
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/bad-drivers/bad-drivers.csv', 'filename':'bad-drivers.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

### Dataset #1 

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


### Dataset #2 

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

**UPDATED**
| Meeting Date | Meeting Time    | Agenda / Discuss at Meeting                                                                                                                                                                                                                            | Next Steps                                                                                                                              |
| ------------ | --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------------------------------------- |
| **2/16**     | 6 PM            | Finalize scope: choose eras/seasons (e.g., 90s/00s/10s or specific season ranges); decide unit of analysis (player-season); decide primary metric (3PA/game vs 3PA/36 vs 3PA rate = 3PA/FGA); confirm position grouping plan (G/F/C or Guard/Wing/Big) | Assign roles (data pull, cleaning, EDA, modeling, writing); set repo structure; draft data dictionary + analysis plan outline           |
| **2/18**     | 6 PM            | Data Checkpoint work session: pull data via `nba_api`; verify joins (player_id ↔ season stats ↔ height); check coverage across eras; start cleaning (height standardization, position parsing, era labeling)                                       | Submit Data Checkpoint; export raw dataset v1 + cleaned dataset v1; document endpoints + code workflow in README                        |
| **2/23**     | 6 PM            | Data cleaning review: handle missing height/position; decide filters (min minutes/games); engineer variables (3PA/36, 3PA rate); sanity checks by era/position (counts, ranges)                                                                        | Lock cleaning rules; produce final analysis-ready dataset; write short “Data & Cleaning” methods paragraph                              |
| **2/25**     | 6 PM            | EDA sprint: scatter/hex plots of height vs 3PA metrics; stratify by era and position; check whether correlation differs by group; identify outliers (very tall high-3PA) and decide treatment                                                          | Finalize EDA notebook + 3–5 key figures; draft EDA narrative answering “overall vs by era/position”                                     |
| **3/01**     | 6 PM            | Modeling plan + first results: run correlations and baseline regressions; test interactions (Height×Era, Height×Position, maybe Height×Era×Position); choose controls (minutes, games; age if available)                                               | Select primary model + backup model; run robustness checks (different metrics, thresholds); prep EDA Checkpoint submission polishing    |
| **3/04**     | 6 PM            | Submit EDA Checkpoint (Due 3/04): clean plots, captions, and clear takeaways; confirm reproducibility (run-all); outline what modeling will prove beyond EDA                                                                                       | Begin full analysis write-up; generate final results tables (effect sizes + CIs); start final visualization polishing                   |
| **3/08**     | 6 PM            | Main analysis + interpretation: finalize models; translate coefficients into plain language (e.g., “+2 inches → X change in 3PA/36 in 90s vs 2010s”); finalize figure set (by-era trend lines, coefficient plot)                                       | Draft Results + Discussion; document limitations (position labeling, pace/era confounds, survivorship); update README + methods details |
| **3/11**     | 6 PM            | Full draft assembly: Methods/Results/Discussion integration; ensure claims match evidence; peer review for clarity + logic; confirm citations and formatting                                                                                           | Produce full project draft v1; polish visuals + captions; clean code and add comments; finalize conclusion                              |
| **3/18**     | Before 11:59 PM | Submission checklist: final run-through of notebook(s), report, figures; verify file names and rubric requirements; confirm all group surveys done                                                                                                     | **Turn in Final Project & Group Project Surveys**                                                                                       |
